In [20]:
import numpy as np
import pandas as pd 
import csv
import matplotlib.pyplot as plt
from matplotlib import*
from pandas import *
from numpy.linalg import inv
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import corner
from IPython.display import display, Math
import emcee


In [59]:
TM=np.random.rand(1500,1500)
R=TM/10e2
sigl=(R+(np.transpose(R)))/2


In [6]:
Data15=pd.DataFrame(np.loadtxt('SN_data_clean.txt', unpack = False))

SN_data_sim1= pd.read_csv('1SN_data_sim1.csv')
dm_prep=SN_data_sim1['mb1']-SN_data_sim1['m_th']
prep15=Data15[5]-Data15[1]
def dm_cut(df,start,end):
    return df.loc[start:end]


#dm_4=np.matrix(dm_cut(dm_prep,484,484+515))


In [9]:
#bin4_signal = np.matrix(np.loadtxt('bin4_signal.txt', unpack = False)) #516x516
#signal8 = np.matrix(np.loadtxt('cleaned_1500_11.dat.txt', unpack = False))#1500x1500
signal_15 = np.matrix(np.loadtxt('cleaned_1500_11.dat.txt', unpack = False))#800x800
#dm8 = np.matrix(np.loadtxt('dm8', unpack = False))
dm15 = np.matrix(dm_cut(prep15,0,1499))
signal_15.shape
#plt.imshow(signal8[:516, :516])
#plt.colorbar()

(1500, 1500)

In [6]:
#print(np.argwhere(np.isnan(signal_15)))
#print(np.argwhere(np.isnan(signal8)))

In [8]:
#I=np.matrix(np.identity(1500))
#c=signal_15+.08**(2)*I
#np.linalg.det(c)
#plt.imshow(c)
#plt.colorbar()

In [14]:
#print(np.__version__)
size=1500
I=np.matrix(np.identity(size))
I.shape
def replaceDiag(matrix, replacementList):
    cp=matrix.copy()
    for i in range(len(replacementList)):
        cp[i,i] = 1/np.sqrt(replacementList[i,0])
    return cp


In [17]:
size=1500
I=np.matrix(np.identity(size))
Q=np.matrix(np.loadtxt('n.txt', unpack = False))
N=replaceDiag(I,Q)
def lnlike(theta,dm,signal,size):
    sigma,A,M= theta
    #I=np.matrix(np.identity(size))
    s2=sigma**2
    C=A*signal+s2*N
    #T1=((A*signal)/s2)
    #T2=((A*signal)/s2)**2
    inv_C=np.linalg.inv(C)
    #print(T2)
    detC=np.linalg.slogdet(C)
    Dm=dm+np.matrix(M*np.ones(size))
    #print(np.matmul(inv_C,np.transpose(Dm)).shape)
    #print(.5*np.matmul(np.transpose(Dm),np.matmul(inv_C,Dm)))
    return ((-.5*detC[1])-(.5*np.matmul(Dm,np.matmul(inv_C,np.transpose(Dm)))))[0,0]



In [16]:
#test
theta=[.08,.6,0]
signal=signal_15
size=1500
v=lnlike(theta,dm15,signal,size)
print(v)

2998.3941869628825


In [18]:
def lnprior(theta):#Prior
    sigma,A,M= theta
    if .005 < sigma < 0.3 and 0< A < 5 and -1 < M < 1:
        return 0.0
    return -np.inf

def lnprob(theta,dm,signal,size):  #prob
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta,dm15,signal_15,size)

In [21]:
ndim, nwalkers = 3, 50
pos = [[.08,.6,0] + [1e-2, 0.2, 0.03]*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(dm15,signal_15,1500))

In [22]:
sampler.run_mcmc(pos,3000)

emcee: Exception while calling your likelihood function:
  params: [ 0.08139546  0.30671504 -0.00642981]
  args: (matrix([[-0.28091435, -0.09451708,  0.03885171, ..., -0.01469889,
          0.09833289,  0.13156344]]), matrix([[ 0.0776449 ,  0.02308193,  0.00667299, ..., -0.0002487 ,
         -0.00024881, -0.00024758],
        [ 0.02308193,  0.04506145,  0.00775092, ..., -0.00018119,
         -0.00018132, -0.00017993],
        [ 0.00667299,  0.00775092,  0.0206198 , ..., -0.00012264,
         -0.00012247, -0.0001218 ],
        ...,
        [-0.0002487 , -0.00018119, -0.00012264, ...,  0.00043286,
          0.00030566,  0.00024327],
        [-0.00024881, -0.00018132, -0.00012247, ...,  0.00030566,
          0.00043272,  0.00022866],
        [-0.00024758, -0.00017993, -0.0001218 , ...,  0.00024327,
          0.00022866,  0.00043255]]), 1500)
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/harperch/anaconda3/lib/python3.6/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-18-54ce5e5ba88c>", line 11, in lnprob
    return lp + lnlike(theta,dm15,signal_15,size)
  File "<ipython-input-17-f73ee2ab412a>", line 14, in lnlike
    detC=np.linalg.slogdet(C)
  File "/home/harperch/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py", line 1817, in slogdet
    sign, logdet = _umath_linalg.slogdet(a, signature=signature)
KeyboardInterrupt


KeyboardInterrupt: 